In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

import os
import numpy as np
from PIL import Image
import glob

from sklearn.preprocessing import LabelEncoder  # label 수로 변환

import splitfolders  # 데이터를 train, val, test폴더로 분리

In [2]:
# 데이터 폴더 경로
path = 'C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split'

In [6]:
def is_error(self, image):
    lines = self.detect_lines_thickness(image)
    line_thickness_threshold = 0.3
    line_cnt = detect_line_count(image)

    if len(lines) >= line_cnt:
        for line in lines:
            if line["thickness"] >= line_thickness_threshold:
                return True
    return False

def detect_line_thickness(image):
    # 이미지 이진화
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY)
    
    # 선의 굵기 측정
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    line_thickness = 0
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        line_thickness = max(line_thickness, w, h)

    return line_thickness


def detect_line_count(image):
    # 이미지 이진화
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY)
    
    # 선 탐지
    edges = cv2.Canny(binary_image, 50, 150)

    # 선의 개수 측정
    lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=50)
    line_count = 0 if lines is None else len(lines)

    return line_count


def process_image(image_path):
    image = cv2.imread(image_path, 0)  # 흑백 이미지로 로드
    
    line_thickness = detect_line_thickness(image)
    line_count = detect_line_count(image)

    if line_thickness >= 0.4 or line_count >= 5:
        # 오류 처리
        # 여기에 오류로 처리하는 로직을 추가하거나, 해당 이미지를 오류 데이터로 분류하는 등의 작업을 수행합니다.
        print(f"Error: {image_path}")
    else:
        # 정상 처리
        # 여기에 정상적으로 처리하는 로직을 추가합니다.
        print(f"Processed: {image_path}")


In [42]:
class BDataset(Dataset):
    def __init__(self, path, transform=None):
        
        self.path = path
        self.transform = transform
        self.image_list = self.load_image_list(path)
    
    
    def load_image_list(self, path, is_training=True):
        image_list = []
        
        # train, validation 폴더의 경우 Broken, Normal폴더로 나뉘어져 있음.
        if is_training:            
            for foldername in os.listdir(path):  # C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split\\  train or val or test
                if foldername == 'Broken' or foldername == 'Normal':
                    folder = os.path.join(path, foldername)   # C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split\\  train or val or test \\ Broken
                    print(folder)
                    # Normal, Broken 이미지
                    for filename in os.listdir(folder):
                        #print(filename)
                        if filename.endswith(".jpg") or filename.endswith(".PNG"):
                            
                            image_path = os.path.join(folder, filename)
                            image_list.append((image_path, filename))  # 레이블 폴더명으로 추가
                            
        # test폴더의 경우 Broken, Normal 폴더가 없고 두 개의 이미지가 혼합되어있음.        
        else:
            for filename in os.listdir(path):
                if filename.endswith(".jpg") or filename.endswith(".PNG"):
                    image_path_test = os.path.join(path, filename)
                    image_list.append(image_path_test)
        return image_list

    
        
    def __len__(self):
        return len(self.image_list)
    
    
    def getitem(self, index):
        if self.is_training:
            image_path, label = self.image_list[index]
            image = Image.open(image_path) # .convert("L")  # 흑백 이미지로 변환
            
            #lines = self.detect_lines(image)
            # 선 굵기와 개수를 기반으로 오류 여부 판단
            #is_error = lines["thickness"] >= 0.4 or lines["count"] >= 6
            
            if self.transform is not None:
                image = self.transform(image)
                
            return image, label

In [ ]:
class BDataset(Dataset):
    def __init__(self, path, transform=None):
        
        self.path = path
        self.transform = transform
        self.image_list = self.load_image_list(path)
    
    
    def load_image_list(self, path, is_training=True):
        image_list = []
        
        # train, validation 폴더의 경우 Broken, Normal폴더로 나뉘어져 있음.
        if is_training:            
            for foldername in os.listdir(path):  # C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split\\  train or val or test
                folder = os.path.join(path, foldername)
                #if (os.listdir(folder)[0] == 'Broken') and (os.listdir(folder)[1] == 'Normal'):   # os.listdir(folder) = ['Broken', 'Normal']
                if foldername == 'Broken' or foldername == 'Normal':
                    folder_br = os.path.join(path, foldername)   # C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split\\  train or val or test \\ Broken
                    folder_nor = os.path.join(path, foldername)  # # C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split\\  train or val or test \\ Normal

                    # Broken 이미지
                    for filename in os.listdir(folder_br):
                        if filename.endswith(".jpg") or filename.endswith(".png"):
                            image_path_br = os.path.join(folder_br, filename)
                            image_list.append((image_path_br, filename))  # 레이블 폴더명으로 추가

                    # Normal 이미지
                    for filename in os.listdir(folder_nor):
                        if filename.endswith(".jpg") or filename.endswith(".png"):
                            image_path_nor = os.path.join(folder_nor, filename)
                            image_list.append((image_path_nor, filename))  # 레이블 폴더명으로 추가
                            
                # test폴더의 경우 Broken, Normal 폴더가 없고 두 개의 이미지가 혼합되어있음.        
                else:
                    for filename in os.listdir(folder):
                        if filename.endswith(".jpg") or filename.endswith(".png"):
                            image_path_test = os.path.join(folder, filename)
                            image_list.append(image_path_test)
        return image_list

    
        
    def __len__(self):
        return len(self.image_list)
    
    
    def getitem(self, index):
        if self.is_training:
            image_path, label = self.image_list[index]
            image = Image.open(image_path) # .convert("L")  # 흑백 이미지로 변환
            
            #lines = self.detect_lines(image)
            # 선 굵기와 개수를 기반으로 오류 여부 판단
            #is_error = lines["thickness"] >= 0.4 or lines["count"] >= 6

            return image, label, is_error
        
        else:
            image_path = self.image_list[index]
            image = Image.open(image_path)
            
            #lines = self.detect_lines(image)
            # 선 굵기와 개수를 기반으로 오류 여부 판단
            #is_error = lines["thickness"] >= 0.4 or lines["count"] >= 6

            return image, is_error
        return image, label

In [ ]:
class BDataset(Dataset):
    def __init__(self, path, transform=None):
        
        self.path = path
        self.transform = transform
        self.image_list = self.load_image_list(path)
    
    
    def load_image_list(self, path, is_training=True):
        image_list = []
        
        # train, validation 폴더의 경우 Broken, Normal폴더로 나뉘어져 있음.
        if is_training:            
            for foldername in os.listdir(path):  # C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split\\  train or val or test

                if (os.listdir(folder)[0] == 'Broken') and (os.listdir(folder)[1] == 'Normal'):   # os.listdir(folder) = ['Broken', 'Normal']
                    folder_br = os.path.join(folder, foldername)   # C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split\\  train or val or test \\ Broken
                    folder_nor = os.path.join(folder, foldername)  # # C:\\Users\\JungHyeona\\Documents\\Project\\AI_deeplearning\\image\\split\\  train or val or test \\ Normal

                    # Broken 이미지
                    for filename in os.listdir(folder_br):
                        if filename.endswith(".jpg") or filename.endswith(".png"):
                            image_path_br = os.path.join(folder_br, filename)
                            image_list.append((image_path_br, filename))  # 레이블 폴더명으로 추가

                    # Normal 이미지
                    for filename in os.listdir(folder_nor):
                        if filename.endswith(".jpg") or filename.endswith(".png"):
                            image_path_nor = os.path.join(folder_nor, filename)
                            image_list.append((image_path_nor, filename))  # 레이블 폴더명으로 추가
                            
                # test폴더의 경우 Broken, Normal 폴더가 없고 두 개의 이미지가 혼합되어있음.        
                else:
                    for filename in os.listdir(folder):
                        if filename.endswith(".jpg") or filename.endswith(".png"):
                            image_path_test = os.path.join(folder, filename)
                            image_list.append(image_path_test)
        return image_list

    
        
    def __len__(self):
        return len(self.image_list)
    
    
    def getitem(self, index):
        if self.is_training:
            image_path, label = self.image_list[index]
            image = Image.open(image_path) # .convert("L")  # 흑백 이미지로 변환
            
            #lines = self.detect_lines(image)
            # 선 굵기와 개수를 기반으로 오류 여부 판단
            #is_error = lines["thickness"] >= 0.4 or lines["count"] >= 6

            return image, label, is_error
        
        else:
            image_path = self.image_list[index]
            image = Image.open(image_path)
            
            #lines = self.detect_lines(image)
            # 선 굵기와 개수를 기반으로 오류 여부 판단
            #is_error = lines["thickness"] >= 0.4 or lines["count"] >= 6

            return image, is_error
        return image, label

    def load_image_list(self):
        image_list = []
        
        folder_br = os.path.join(self.path, 'Broken')  # 에러 이미지 경로
        folder_nor = os.path.join(self.path, 'Normal')  # 정상 이미지 경로
        
        for filename in os.listdir(folder_br):
            if filename.endswith(".png"):
                img_path_br = os.path.join(folder_br, filename)
                img_path_nor = os.path.join(folder_nor, filename)
                
                if os.path.isfile(img_path_nor):
                    image_list.append((img_path_br, img_path_nor))
        return image_list
    
    
    def __getitem__(self, index):
        img_path = self.img_list[index]
        label = self.label_list[index]
        
        image = Image.open(img_path)

        if self.transform is not None:
            image = self.transform(image)

## 이미지 평균 및 표준편차

In [38]:
# 데이터 전처리
transform = transforms.Compose([
                transforms.ToTensor()
])

In [41]:
trainset = BDataset(path + "\\train", transform)

C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\train\Broken
C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\train\Normal


NotImplementedError: 

In [9]:
train_set = BDataset(path + "\\train", transform)
val_set = BDataset(path + "\\val", transform)
test_set = BDataset(path + "\\test", transform)

### 결과 -> 해당 결과의 경우 train, val, test별로의 평균값이 아닌 파일을 나누기 전 Normal, Broken을 가져왔을 때 돌려본 결과.

### normal : 1000개
shape: (1000, 3, 808, 1182)<br>
min: (0.0, 0.05882353, 0.019607844)<br>
max: (0.92156863, 0.92156863, 0.92941177)<br>
mean: (0.11181334, 0.111890435, 0.1135668)<br>
std: (0.026530448, 0.02637402, 0.02658573)

### broken
shape: (523, 3, 808, 1182)<br>
min: (0.0, 0.05882353, 0.019607844)<br>
max: (0.92156863, 0.92156863, 0.92941177)<br>
mean: (0.11188801, 0.111966185, 0.11256382)<br>
std: (0.03245135, 0.032322615, 0.03192493)

In [ ]:
# 이미지의 RGB 채널별 통계량 확인 함수
def print_stats(dataset):
    imgs = np.array([img.numpy() for img, _ in dataset])
    print(f'shape: {imgs.shape}')
    
    min_r = np.min(imgs, axis=(2, 3))[:, 0].min()
    min_g = np.min(imgs, axis=(2, 3))[:, 1].min()
    min_b = np.min(imgs, axis=(2, 3))[:, 2].min()

    max_r = np.max(imgs, axis=(2, 3))[:, 0].max()
    max_g = np.max(imgs, axis=(2, 3))[:, 1].max()
    max_b = np.max(imgs, axis=(2, 3))[:, 2].max()

    mean_r = np.mean(imgs, axis=(2, 3))[:, 0].mean()
    mean_g = np.mean(imgs, axis=(2, 3))[:, 1].mean()
    mean_b = np.mean(imgs, axis=(2, 3))[:, 2].mean()

    std_r = np.std(imgs, axis=(2, 3))[:, 0].std()
    std_g = np.std(imgs, axis=(2, 3))[:, 1].std()
    std_b = np.std(imgs, axis=(2, 3))[:, 2].std()
    
    print(f'min: {min_r, min_g, min_b}')
    print(f'max: {max_r, max_g, max_b}')
    print(f'mean: {mean_r, mean_g, mean_b}')
    print(f'std: {std_r, std_g, std_b}')

In [ ]:
print_stats(dataset)

### 데이터 전처리

In [ ]:
# 데이터 전처리
transform = transform = transforms.Compose([
                transforms.Resize((512, 512)),
                transforms.RandomHorizontalFlip(),   # 좌우반전
                # transforms.RandomResizedCrop((60, 60)),   # (w, h). 이미지에서 잘라낼 크기 설정하면, 크기만큼 랜덤으로 잘린다.
                # transforms.RandomErasing(),      # 이미지에서 랜덤하게 박스 모양으로 지운다. PIL image에 바로 적용 불가 -> tensor 필요
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])

In [43]:
class Dataloader():
    def __init__(self, path):
        #self.path = path
        normalize = transforms.Normalize((0.111850675, 0.11192831, 0.11306531), (0.029490899, 0.0293483175, 0.029255329999999996))
        
        transform_train = transforms.Compose([
                        transforms.Resize((512, 512)),
                        #transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        normalize
                        ])
        transform_test = transforms.Compose([
                        transforms.Resize((512, 512)),
                        transforms.ToTensor(),
                        normalize
                        ])
        
        trainset = BDataset(path + "\\train", transform_train)
        valset = BDataset(path + "\\val", transform_train)
        testset = BDataset(path + "\\test", transform_test)
        
        train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
        val_loader = DataLoader(valset, batch_size=64, shuffle=True, num_workers=4)
        test_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4)
        
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        
    def getloader(self):
        return self.train_loader, self.val_loader#, self.test_loader

In [44]:
train_loader = Dataloader(path)
val_loader = Dataloader(path)
#test_loader = Dataloader(path)

C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\train\Broken
C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\train\Normal
C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\val\Broken
C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\val\Normal
C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\train\Broken
C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\train\Normal
C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\val\Broken
C:\Users\JungHyeona\Documents\Project\AI_deeplearning\image\split\val\Normal


## 모델

In [45]:
class CNN(nn.Module):
    
    def __init__(self):
        
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            # (512, 512, 3) -> (32, 32, 32), (16, 16, 32)
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.BatchNorm2d(64),    #batch layer
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.BatchNorm2d(64),    #batch layer
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.BatchNorm2d(64),    #batch layer

            nn.MaxPool2d(kernel_size=2),    #pooling layer

            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.BatchNorm2d(64),    #batch layer
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.BatchNorm2d(64),    #batch layer
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.BatchNorm2d(64),    #batch layer

            nn.MaxPool2d(kernel_size=2),    #pooling layer

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  #cnn layer
            nn.BatchNorm2d(128),    #batch layer
            nn.Flatten(),
            nn.Linear(4*4*128, 18),
            nn.Linear(18, 2)  # 수정 필요한 부분: 출력 뉴런 수
        )


    def forward(self, x):
        out = self.model(x)
        return out

In [46]:
model = CNN()

# GPU 사용 가능 여부에 따라 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
model.to(device)

Using cuda device


CNN(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=

In [47]:
loss_func = nn.CrossEntropyLoss()
learning_rate = 0.0002
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

In [48]:
for X, Y in train_loader:
    print(X)

TypeError: 'Dataloader' object is not iterable

In [49]:
#total_batch = len(train_loader)
#print('총 배치의 수 : {}'.format(total_batch))

for epoch in range(10):
    avg_cost = 0

    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss_func(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

TypeError: 'Dataloader' object is not iterable

In [ ]:
# 모델 평가 함수
def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# 모델 평가
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
val_accuracy = evaluate(model, val_loader, device)
print(f'Validation Accuracy: {val_accuracy:.2f}%')

In [ ]:
# 모델 저장
torch.save(model, path+'B_cnn.pt')